# Clusteranalyse mit K-Means

Die Clusteranalyse ist eine Art des unüberwachten Lernens. Es liegt hierbei eine Datenmenge vor, die aus mehreren Merkmalen besteht, anders als beim überwachten Lernen gibt es allerdings kein durch die Fragestellung gegebenes Zielmerkmal. 

Ziel einer Clusteranalyse ist es, Gruppen zu finden, die ähnliche Eigenschaften besitzen, sogenannte Cluster. Manchmal ist es nützlich, für jedes Cluster typische Repräsentanten angeben zu können, dies sind dann die Clusterzentren.

Ein möglicher Anwendungsbereich von Clusteranalysen ist beispielsweise die Analyse sozialer Netzwerke bezüglich ihrer Nutzer:innen. 


<hr style="border:1px solid gray"> </hr>

## Inhalt

1. [Künstliche Erzeugung der Datenmenge](#kap1)


2. [k-Means](#kap2)  
    2.1 [Durchführung](#kap21)  
    2.2 [Visualisierung](#kap22)  
    2.3 [Gütemaß Inertia](#kap23)  
    2.4 [Bestimmung von k - die Ellenbogen-Methode](#kap24)  

3. [Fazit](#kap3) 
    

<hr style="border:1px solid gray"> </hr>

## 1. Künstliche Erzeugung der Datenmenge <a name="kap1"></a>

Für eine Demonstration unterschiedlicher Arten von Clusteranalysen wird im Folgenden ein künstlicher Datensatz erzeugt.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn import cluster, datasets, mixture

Für die Erzeugen von Clustern von Punkten gibt es in scikit-learn eine eigene Funktion `make_blobs`:

In [ ]:
blob_centers = np.array(
    [[1,  2],
     [4 , 5],
     [3,  1],
     [2.2,  2.8],
     ])
blob_std = np.array([0.3, 0.2, 0.4, 0.2])

In [ ]:
X, y = datasets.make_blobs(n_samples=500, centers=blob_centers, cluster_std=blob_std, random_state=7)  
print(X[0:5,:])
print(y[0:5])

Zunächst werden für die Cluster Mittelpunkte vorgegeben. Dann werden Zufallspunkte um diese Mittelpunkte herum erzeugt. Neben den Mittelpunkten wird auch angegeben, wie stark die Zufallspunkte darum herum streuen. Dies wird durch den Parameter für die Standardabweichungen `cluster_std=blob_std` angegeben.

In `y` ist abgespeichert, zu welcher Blase der zugehörige Punkt von `X` gehört. Auf diese Information wird später nicht mehr zurückgegriffen! Beim unüberwachten Lernen gibt es keine Klassenzugehörigkeit.

Um unsere Datensätze schön zu plotten, wird die folgende Funktion definiert.

In [ ]:
def plot_clusters(X):
    plt.scatter(X[:, 0], X[:, 1])
    plt.xlabel("$x_1$", fontsize=14)
    plt.ylabel("$x_2$", fontsize=14, rotation=0, labelpad=20)
    plt.show()

Damit können nun die künstlich erzeugten Cluster visualisiert werden:

In [ ]:
plot_clusters(X)

Zu sehen sind also links zwei dicht beieinanderliegende Cluster, die sich nur schlecht unterscheiden lassen, ein weiterer Cluster oben rechts, und ein etwas weniger dichter Cluster unten rechts.
Das Ziel der Clusteranalyse ist es, diese Cluster zuverlässig zu erkennen.

<hr style="border:1px solid gray"> </hr>

## 2. k-Means <a name="kap2"></a>

### 2.1 Durchführung <a name="kap21"></a>

Das k-Means-Verfahren beginnt mit der Festlegung der Anzahl der Cluster: k. Anschließend werden folgende Berechnungsschritte durchgeführt:
- k-Means wählt zunächst zufällig k Punkte aus dem Datensatz aus und setzt diese als vorläufige Clustermittelpunkte. 

- Jetzt werden die Cluster gebildet, indem jeder Punkt dem nächstgelegenen Clustermittelpunkt zugeordnet wird. Da die Clustermittelpunkte im ersten Schritt zufällig gewählt wurden, sind die Cluster noch sehr unausgewogen und zeigen noch nicht das gewünschte Ergebnis. 

- Aus diesen Clustern werden neue Zentren ermittelt, indem die Mittelwerte der Clusterpunkte berechnet werden. (Diese neuen Mittelpunkte müssen nicht unbedingt Punkte des Datensatzes sein!) 

- Mit diesen neuen Mittelpunkten wird das Verfahren wiederholt wodurch wieder neue Cluster entsehen. 

- Das Vorgehen wird solange wiederholt, bis sich die gebildeten Cluster nicht mehr ändern.

Das k-Means-Verfahren wird nun für die künstlich erzeugte Punktemenge durchgefüht. Die Syntax von scikit-learn ist beim unüberwachten Lernen ganz ähnlich zu der beim überwachten Lernen: Das Modell wird zunächst initialisiert und dann gefittet.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
k = 4
kmeans = KMeans(n_clusters=k, random_state=0)

y_pred = kmeans.fit_predict(X)
print("Anzahl der Iterationen:", kmeans.n_iter_)

Die Ausgabe zeigt, dass das Verfahren bereits nach 2 Iterationsschritten endet. Das Ergebnis ist bereits stabil. 

### 2.2 Visualisierung <a name="kap22"></a>

Zur Darstellung der Ergebnisse wird jeder Punkt einem Cluster zugeordnet (die Cluster sind von 0 bis k-1 durchnummeriert) und anschließend eine Funktion zur Visualisierung der Cluster ausgeführt:

In [ ]:
y_pred

In [ ]:
def plot_kmeans(X, mod):                                     # mod ist das trainierte Modell
    plt.scatter(X[:,0], X[:,1], c=mod.labels_.astype(float)) # mod.labels_ ist dasselbe wie y_pred
    plt.scatter(mod.cluster_centers_[:, 0], mod.cluster_centers_[:, 1], marker='x', s=2, linewidths=12, color='k')
    plt.xlabel("$x_1$", fontsize=14)
    plt.ylabel("$x_2$", fontsize=14, rotation=0, labelpad=20)
    plt.show()

In [ ]:
print(kmeans.cluster_centers_)
plot_kmeans(X, kmeans)

Die Cluster werden also sehr gut erkannt, und tatsächlich liegen die erhaltenen Clustermittelpunkte sehr nah an den oben festgelegten ``blob_centers``.

### 2.3 Gütemaß Inertia <a name="kap23"></a>

Bisher wurde die Güte einer Clusterung nur visuell überprüft. Um sie maschinell zu prüfen, wird ein Maß benötigt, dass die Güte als Zahl ausdrückt. 

*Inertia* (Trägheit) oder *within-cluster sum of squares (wcss)* (dt. Summe der Quadrate innerhalb der Cluster) ist ein Gütemaß, das sich aus der Summe der quadratischen Abweichungen der Punkten zu ihren Clustermittelpunkten errechnet. Je kleiner der Wert, desto besser ist die Clusterung. Scikit-learn berechnet den Inertia-Wert automatisch, er kann mit `kmeans.inertia_` aufgerufen werden:

(Um die Inertia-Werte der bisher betrachteten Situationen besser einordnen zu können, werden die Cluster erneut als Plot ausgegeben.)

In [ ]:
print("Inertia bei Beispiel:", kmeans.inertia_)
plot_kmeans(X, kmeans)

Nun werden als Startwerte nur Werte gewählt, die im oberen rechten Cluster liegen und dann erneut eine Clusteranalyse vorgenommen:

In [ ]:
startwerte = np.array([[3.6, 4.6], [3.8, 5.2],[4.2, 4.9],[4, 4.9]])

In [ ]:
k = 4
kmeans_schlecht = KMeans(n_clusters=k, init=startwerte, n_init=1, random_state=0).fit(X)
print("Anzahl Iterationen:", kmeans_schlecht.n_iter_)

Bei schlecht gewählten Startwerten sind mehr Iterationsschritte nötig. Hier sind es 4 Schritte im Vergleich von 2 oben. 

In [ ]:
print("Inertia bei den schlecht gewählten Startwerten:", kmeans_schlecht.inertia_)
plot_kmeans(X, kmeans_schlecht)

Zu sehen ist, dass in diesem Beispiel das obere Cluster in drei aufgeteilt wird, während die übrigen Datenpunkte einem großen Cluster zugeordnet werden. Der Inertia-Wert ist nun bedeutend schlechter!

### 2.4 Bestimmung von k &ndash; die Ellenbogen-Methode <a name="kap24"></a>

Bisher wurde die Anzahl der zu bestimmenden Cluster k immer graphisch anhand eines Plots der Punktemenge ausgewählt. Doch wie ist das Vorgehen, wenn es mehr als zwei bzw. drei Merkmale gibt? In diesem Fall ist es nicht möglich, am Plot zu sehen, wie viele Cluster gebildet werden sollten, also wie das k zu wählen ist.

Eine Idee könnte sein, verschiedene Werte von k auszuprobieren, und denjenigen mit dem besten (d.h. kleinsten) Inertia-Wert zu wählen. Dieses Vorgehen ist allerdings problematisch, da Inertia mit steigendem k automatisch kleiner wird, unabhängig davon, ob das gewählte k sinnvoll ist oder nicht. Intuitiv lässt sich das einfach erklären: Wenn k als die Anzahl der Punkte im Datensatz gewählt wird, dann bildet jeder Punkt ein eigenes Cluster, und der Abstand zum Clustermittelpunkt ist 0, und damit ist auch Inertia gleich 0. Ein kleiner Intertia-Wert alleine ist also nicht aussagekräftig. 

Dennoch kann Inertia helfen: Die Betrachtung der Veränderung von Inertia in Abhängigkeit von k führt in den meisten Fällen zu einem Verlauf, bei dem Inertia bei der richtigen Wahl von k stark und danach nur noch schwach abnimmt. Der entstehende "Ellenbogen" wird in der folgenden Kurve, in der für jedes k der Inertia-Wert für den ursprünglichen Datensatz an Clustern abgetragen wird, dargestellt:

In [ ]:
inertias = []
number_clusters = range(1,8)

for i in number_clusters:
    kmeans_i = KMeans(i, random_state=0).fit(X)
    inertias.append(kmeans_i.inertia_)
    print("Inertia für k =",i,"ist:",inertias[i-1])

plt.plot(number_clusters, inertias);
plt.title('Ellenbogenkurve')
plt.xlabel('Anzahl der Cluster ("k")', labelpad=10)
plt.ylabel('Inertia', labelpad=10)
plt.show()

Bis zum Wert 3 nimmt die Inertia deutlich ab. Von 3 auf 4 ist noch einmal eine größere Verbesserung erkennbar. Ab einem Wert von k=4 gibt es nur noch eine geringe Änderung. Daher ist k=4 der günstigste Wert (der Punkt am Ende des "Ellenbogens"). 

Das k-Means-Verfahren funktioniert am besten, wenn die gesuchten Cluster eine "runde" Form haben. 
In Daten treten jedoch auch komplexere Strukturen auf. Dann liefert k-Means keine so guten Ergebnisse mehr. In diesem Fall können andere Verfahren (vgl. Modul 7) eingesetzt werden.  

<div class="alert alert-block alert-success">
<b>Arbeitsaufträge:</b>   
    
1. Die in Kapitel 2.2 in lila und gelb dargestellen Cluster liegen nah beieinander. Verändern Sie die Parameter des k-Means so, dass diese beiden Cluster zu einem zusammengefasst werden.

2. Nutzen Sie die Funktion `make_blobs`, um eine künstliche Punktemenge mit 6 Zentren zu erzeugen und visualisieren Sie diese.  
Wenden Sie auf die so erzeugte Punktemenge das kmeans-Verfahren an und geben Sie einen Plot der Clusterung aus.

</div>

In [ ]:
# Platz für Arbeitsauftrag 1

In [ ]:
# Platz für Arbeitsauftrag 2

## 3. Fazit <a name="kap3"></a>

Die Anwenudung des K-Means mit k=6 zeigt, dass fast alle Punkte dem Cluster geordnet werden, von dem sie auch erzeugt wurden. 

Es wurde das k-Means Verfahren auf einen selbsterzeugten Beispieldatensatz angewendet. Hierbei wurde...

- gezeigt, dass die Umsetzung des Verfahrens der Syntax Logik der überwachten Lernverfahren entspricht, 

- der Einfluss des Hyperparameters k und der Auswahl der Startwerte gezeigt, 

- die Bestimmung des Gütemaßes Inertia und die Durchführung der Ellenbogenmethode thematisiert. 